# The Diet Problem

The Diet Problem was motivated by the Army's desire to meet the nutritional requirements of its soldiers while minimizing the cost. Here, we solve the diet problem with real data.


## Part Three:	Low-cholesterol diet model
Here, we focus on minimizing cholesterol intake, and solve a full-sized problem.


# Solution

Before presenting the codes and analysis, I want to explain the chalnnges which I faced to solve this problem:

1. We have to take into account that there is no minimum or maximum daily intake provided for for Cholesterol, Fatty acids total trans, and Fatty acids-total saturated. I really do not know if this is just not provided to us or it is really a medical fact. But in this problem, I did not write any constraints for min and max for these 3 variables. 
   + Obviously the general constaraints of **>=0** for every food is applied here, too.
   + The objective funtion is to **minimize Cholesterol**. So I did not need to worry about maximum intake. 


2. There are lots of missing value for 'Fatty acids total trans'. **I replaced all missing values with 0**. I believe that doesnt impact the final solution becasue a) there is no constraints specific to 'Fatty acids total trans'. b) There is high corrolation between Cholesterol, Fatty acids total trans, and Fatty acids-total saturated. So when We are trying to minimize the Cholesterol, other also will be reduced (if not minimized).

3. There was two columns for energy. One culmns was 4 times the other one. I could not find what is the difference between the two. I used the sammler value, and dropped the other one.


In [1]:
from pulp import *
import pandas as pd

# 1. Read and explore the data

Here, we load the data from excel sheet, and explore the data

In [2]:
# load the diet data
df = pd.read_excel( open('data/diet_large.xls','rb'),sheet_name='Sheet1', header=1)
del df['Energy.1']
df.head()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,"Butter, salted",0.85,0.06,717,15.87,24,0.02,2,24,24,...,0.034,0.042,0.11,0.003,3,0.17,7,215.0,NaN,51.368
1,"Butter, whipped, with salt",0.85,0.06,717,15.87,24,0.16,2,23,26,...,0.034,0.042,0.11,0.003,3,0.13,7,219.0,NaN,50.489
2,"Butter oil, anhydrous",0.28,0,876,0.24,4,0,0,3,5,...,0.005,0.003,0.01,0.001,0,0.01,8.6,256.0,NaN,61.924
3,"Cheese, blue",21.4,2.34,353,42.41,528,0.31,23,387,256,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75.0,NaN,18.669
4,"Cheese, brick",23.24,2.79,371,41.11,674,0.43,24,451,136,...,0.351,0.118,0.288,0.065,20,1.26,2.5,94.0,NaN,18.764


# 2. Clean the Data

We just explore the data, and make sure the format and values are correct. Here, we noticed that the last 3 rows of xls file contains one empty row, minimum daily intake, and maximum daily intake

In [3]:
df.tail()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7145,"Turtle, green, raw",19.8,0,89,78.5,118,1.4,20,180,230,...,0.15,1.1,NaN,0.12,15,1,0.1,50.0,NaN,0.127
7146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7147,NaN,56,130,2400,3700,1000,8,270,700,4700,...,1.3,16,5,1.3,400,2.4,120,NaN,NaN,NaN
7148,NaN,g/d,g/d,kcal,g,mg/d,mg/d,mg/d,mg/d,mg,...,mg/d,mg/d,mg/d,mg/d,microg/d,microg/d,microg/d,NaN,NaN,NaN
7149,NaN,1000000,1000000,1000000,1000000,2500,45,400,4000,1000000,...,1000000,35,1000000,100,1000,1000000,1000000,NaN,NaN,NaN


In [4]:
# data cleaning
data = df[0:7146].fillna(0).copy()
data.tail()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P","Potassium, K",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7141,"Frog legs, raw",16.4,0.0,73,81.9,18,1.5,20,147,285,...,0.25,1.2,0.0,0.12,15,0.4,0.1,50.0,0.0,0.076
7142,"Fish, mackerel, salted",18.5,0.0,305,43.0,66,1.4,60,254,520,...,0.19,3.3,0.0,0.41,15,12.0,7.8,95.0,0.0,7.148
7143,"Mollusks, scallop, (bay and sea), cooked, steamed",23.2,0.0,112,73.1,115,3.0,55,338,476,...,0.06,1.3,0.0,0.10,12,1.3,0.2,53.0,0.0,0.146
7144,"Mollusks, snail, raw",16.1,2.0,90,79.2,10,3.5,250,272,382,...,0.12,1.4,0.0,0.13,6,0.5,0.1,50.0,0.0,0.361
7145,"Turtle, green, raw",19.8,0.0,89,78.5,118,1.4,20,180,230,...,0.15,1.1,0.0,0.12,15,1.0,0.1,50.0,0.0,0.127


# 3. Data Prepration

Here, we orgonize the data into more readible formats. 

+ We create some dictionaries for each listed food item and its corresponding nutrition value.
+ We create two dictionaries for Minimum and Maximum daily intake
+ We create a list of name of the foods and nutritions

In [5]:
foods_dict = data.set_index('Long_Desc').to_dict()
# Sample output
print("\n\nCholesterol\n", list(foods_dict['Cholesterol'].items())[0:10], "...")



Cholesterol
 [('Butter, salted', 215.0), ('Butter, whipped, with salt', 219.0), ('Butter oil, anhydrous', 256.0), ('Cheese, blue', 75.0), ('Cheese, brick', 94.0), ('Cheese, brie', 100.0), ('Cheese, camembert', 72.0), ('Cheese, caraway', 93.0), ('Cheese, cheddar', 105.0), ('Cheese, cheshire', 103.0)] ...


In [6]:
# Minimum daily intake
dmin = df.iloc[7147,1:-3].to_dict()
# Maximum daily intake
dmax = df.iloc[7149,1:-3].to_dict()

print("Minimum daily intake:\n", dmin)

print("\n\n Maximum daily intake:\n", dmax)

Minimum daily intake:
 {'Protein': 56, 'Carbohydrate, by difference': 130, 'Energy': 2400, 'Water': 3700, 'Calcium, Ca': 1000, 'Iron, Fe': 8, 'Magnesium, Mg': 270, 'Phosphorus, P': 700, 'Potassium, K': 4700, 'Sodium, Na': 1500, 'Zinc, Zn': 11, 'Copper, Cu': 0.9, 'Manganese, Mn': 2.3, 'Selenium, Se': 55, 'Vitamin A, RAE': 900, 'Vitamin E (alpha-tocopherol)': 15, 'Vitamin D': 200, 'Vitamin C, total ascorbic acid': 90, 'Thiamin': 0.0012, 'Riboflavin': 1.3, 'Niacin': 16, 'Pantothenic acid': 5, 'Vitamin B-6': 1.3, 'Folate, total': 400, 'Vitamin B-12': 2.4, 'Vitamin K (phylloquinone)': 120}


 Maximum daily intake:
 {'Protein': 1000000, 'Carbohydrate, by difference': 1000000, 'Energy': 1000000, 'Water': 1000000, 'Calcium, Ca': 2500, 'Iron, Fe': 45, 'Magnesium, Mg': 400, 'Phosphorus, P': 4000, 'Potassium, K': 1000000, 'Sodium, Na': 2300, 'Zinc, Zn': 40, 'Copper, Cu': 10, 'Manganese, Mn': 11, 'Selenium, Se': 400, 'Vitamin A, RAE': 3000, 'Vitamin E (alpha-tocopherol)': 1000, 'Vitamin D': 2000, 

In [7]:
foods = list(data['Long_Desc'])
constraints = list(foods_dict.keys())[1:]

print("foods:\n", foods[0:20], "...")

print("\n\nconstraints:\n", constraints[0:20], "...")


foods:
 ['Butter, salted', 'Butter, whipped, with salt', 'Butter oil, anhydrous', 'Cheese, blue', 'Cheese, brick', 'Cheese, brie', 'Cheese, camembert', 'Cheese, caraway', 'Cheese, cheddar', 'Cheese, cheshire', 'Cheese, colby', 'Cheese, cottage, creamed, large or small curd', 'Cheese, cottage, creamed, with fruit', 'Cheese, cottage, nonfat, uncreamed, dry, large or small curd', 'Cheese, cottage, lowfat, 2% milkfat', 'Cheese, cottage, lowfat, 1% milkfat', 'Cheese, cream', 'Cheese, edam', 'Cheese, feta', 'Cheese, fontina'] ...


constraints:
 ['Carbohydrate, by difference', 'Energy', 'Water', 'Calcium, Ca', 'Iron, Fe', 'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K', 'Sodium, Na', 'Zinc, Zn', 'Copper, Cu', 'Manganese, Mn', 'Selenium, Se', 'Vitamin A, RAE', 'Vitamin E (alpha-tocopherol)', 'Vitamin D', 'Vitamin C, total ascorbic acid', 'Thiamin', 'Riboflavin', 'Niacin'] ...


# 4. Define the optimization problem framework

To define the optimization problem, we need to define the variables, constraines and objective funtions. 

Note: LpMinimize (default)

In [8]:
# create the minimization problem framework 
prob3 = LpProblem('PuLPTutorial', LpMinimize)

In [9]:
# Define a dictionary for foods referenced Variables (# of serving)
food_vars = LpVariable.dicts('Food', foods, lowBound=0, cat='Continuous')

In [10]:
# Define a dictionary for foods -  (chosen or not)
chosen_Vars = LpVariable.dicts("Chosen", foods, lowBound=0, upBound=1, cat="Binary")

In [11]:
# define the objective function: Cost of Foods per person
prob3 += lpSum([foods_dict['Cholesterol'][f] * food_vars[f] for f in foods])

In [12]:
# Add maximum and minimum daily nutrition constraints
for constraint in constraints[:-3]:
    prob3 += lpSum([foods_dict[constraint][f] * food_vars[f] for f in foods]) >= dmin[constraint], constraint + "minRequirement"
    prob3 += lpSum([foods_dict[constraint][f] * food_vars[f] for f in foods]) <= dmax[constraint], constraint + "MaxRequirement"
    

In [13]:
# Write down the the problem data
#prob3.writeLP("DietModelforProblem3.lp")

# 5. Solve the optimization and print the calulated values

Here, we solve the optimization problem, and then print the calculated **Non-Zero** values.

In [14]:
# Solve the problem
prob3.solve()

# Print the solution
print ("Status:", LpStatus[prob3.status])

Status: Optimal


In [15]:
# Print the calculated optimum serving for each food
for serving in prob3.variables():
    if serving.varValue > 0:
        print (serving.name, " : ", serving.varValue)

Food_Cereals,_QUAKER,_Instant_Oatmeal,_NUTRITION_FOR_WOMEN,_Vanilla_  :  0.123614
Food_Cereals_ready_to_eat,_GENERAL_MILLS,_CINNAMON_TOAST_CRUNCH  :  0.0385779
Food_Cranberries,_high_bush,_raw_(Alaska_Native)  :  0.505488
Food_Fiddlehead_ferns,_raw  :  0.581716
Food_Infant_formula,_ROSS,_ISOMIL,_with_iron,_powder,_not_reconstitu  :  0.493829
Food_Mushrooms,_straw,_canned,_drained_solids  :  2.67131
Food_Oil,_bearded_seal,_(oogruk_oil)_(Alaska_Native)  :  0.244672
Food_Oil,_vegetable,_cupu_assu  :  1.33419
Food_Peppers,_hungarian,_raw  :  0.407096
Food_Radicchio,_raw  :  4.03361
Food_Radishes,_oriental,_dried  :  0.299571
Food_Sauce,_barbeque,_low_sodium,_commercially_canned  :  1.65649
Food_Snacks,_CHEX_mix  :  0.0182453
Food_Soup,_chicken_broth,_low_sodium,_canned  :  4.77935
Food_Soy_protein_isolate,_PROTEIN_TECHNOLOGIES_INTERNATIONAL,_SUPRO  :  0.379705
Food_Tea,_herb,_chamomile,_brewed  :  9.2954
Food_Water,_bottled,_generic  :  14.2284


In [16]:
# Print the minimum Cost Per Person    
print ("\n\nCholesterol intake = ", value(prob3.objective))



Cholesterol intake =  0.0
